In [0]:
!pip install -q tensorflow-gpu==2.0.0-rc1

In [0]:
!pip install tf-nightly-gpu

In [0]:
# mount to your google drive
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import os
import numpy as np
import math
import matplotlib.pyplot as plt

print(tf.__version__)

2.2.0-dev20200304


In [0]:
layer_in_block = {'vgg11': [1, 1, 2, 2, 2],
                  'vgg13': [2, 2, 2, 2, 2],
                  'vgg16': [2, 2, 3, 3, 3],
                  'vgg19': [2, 2, 4, 4, 4]}


class VggConv(tf.keras.layers.Layer):
  def __init__(self, filter_num=None, kernel_size=(3, 3), \
               activation='relu', padding='same', kernel_initializer='he_normal'):
    super(VggConv, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                        kernel_size=kernel_size,
                                        activation=activation,
                                        padding=padding,
                                        kernel_initializer=kernel_initializer)   
    
  def call(self, inputs, training=None):
    x = self.conv1(inputs)
    return x

class VggConvs(tf.keras.layers.Layer):
  def __init__(self, pool_size=(2, 2)):
    super(VggConvs, self).__init__()
    self.bn1 = tf.keras.layers.BatchNormalization()
    
    self.pool1 = tf.keras.layers.MaxPool2D(pool_size=pool_size)
  
  def call(self, inputs, training=None):
    x = self.bn1(inputs, training=training)
    x = self.pool1(x)
    return x

class VggDense(tf.keras.layers.Layer):
  def __init__(self, filter_num=None, classes=1):
    super(VggDense, self).__init__()
    self.Flatten = tf.keras.layers.Flatten()
    self.Dense1 = tf.keras.layers.Dense(filter_num, activation='relu')
    self.bn1 = tf.keras.layers.BatchNormalization()
    if classes == 1:
      self.Dense2 = tf.keras.layers.Dense(classes, activation=tf.keras.activations.sigmoid)
    else:
      self.Dense2 = tf.keras.layers.Dense(classes, activation=tf.keras.activations.softmax)

  def call(self, inputs):
    x = self.Flatten(inputs)
    x = self.Dense1(x)
    x = self.bn1(x)
    x = self.Dense2(x)

    return x


def vgg_convs_layer(filter_num=None, blocks=None,  kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer='he_normal', pool_size=(2, 2)):
  vgg_block = tf.keras.Sequential()
  for i in range(blocks):
    vgg_block.add(VggConv(filter_num=filter_num,  kernel_size=kernel_size, activation=activation, padding=padding, kernel_initializer=kernel_initializer))
  vgg_block.add(VggConvs(pool_size=pool_size))
  return vgg_block

def vgg_dense_layer(filter_num=None, classes=1):
  vgg_block = tf.keras.Sequential()
  vgg_block.add(VggDense(filter_num=filter_num, classes=classes))
  return vgg_block

class VggNet(tf.keras.Model):
  def __init__(self, include_top=True, layer='vgg16'):
    super(VggNet, self).__init__()
    self.include_top = include_top
    self.conv1 = vgg_convs_layer(filter_num = 64, blocks = layer_in_block[layer][0])
    self.conv2 = vgg_convs_layer(filter_num = 128, blocks =  layer_in_block[layer][1])
    self.conv3 = vgg_convs_layer(filter_num = 256, blocks = layer_in_block[layer][2])
    self.conv4 = vgg_convs_layer(filter_num = 512, blocks = layer_in_block[layer][3])
    self.conv5 = vgg_convs_layer(filter_num = 512, blocks = layer_in_block[layer][4])
    self.dense = VggDense(filter_num = 256, classes = 1)

  def call(self, inputs):
    x = self.conv1(inputs)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x = self.conv5(x)
    if self.include_top:
      x = self.dense(x)
    
    return x

def vgg11():
  return VggNet(include_top=True, layer='vgg11')

def vgg13():
  return VggNet(include_top=True, layer='vgg13')

def vgg16():
  return VggNet(include_top=True, layer='vgg16')

def vgg19():
  return VggNet(include_top=True, layer='vgg19')

In [0]:
vgg16 = vgg16()

In [0]:
def print_model_summary(network):
    network.build(input_shape=(None, IMG_HEIGHT, IMG_WIDTH, 3))
    network.summary()

print_model_summary(network=vgg16)

Model: "vgg_net_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_5 (Sequential)    multiple                  38976     
_________________________________________________________________
sequential_6 (Sequential)    multiple                  221952    
_________________________________________________________________
sequential_7 (Sequential)    multiple                  1476352   
_________________________________________________________________
sequential_8 (Sequential)    multiple                  5901824   
_________________________________________________________________
sequential_9 (Sequential)    multiple                  7081472   
_________________________________________________________________
vgg_dense_1 (VggDense)       multiple                  6424065   
Total params: 21,144,641
Trainable params: 21,141,185
Non-trainable params: 3,456
_________________________________________